In [4]:
import math
import numpy as np
from matplotlib import pyplot as plt

# Постановка задачи

В рамках данной работы выполняются следующие задачи:
<ol>
<li>Определение функции (CDF) и плотности (PDF)  синтетического распределения Z=X+Y, причем X~Triang(0,2) и Y~Exp(0.25).</li>
<li>Генерация выборки для данного синтетического распределения Z объемом 10 000 элементов</li>
<li>Построение распределения выборочных средних объема 1, 3, 5, 7 элементов</li>
</ol>

<h3>1. Определение CDF и PDF синтетиского распределения</h3>

Введем определение исходных распределений:

${Z=X + Y}, \quad 
{X \thicksim Triang\left(0,2\right)\quad Y \thicksim Exp\left(\frac{1}{4}\right) }
$

${f_x =
  \begin{cases}
    0    & , x < 0\\
    x    & , 0\leq x \leq 1\\
    2-x  & , 1\leq x \leq2\\
    0    & , 0 < x\\
  \end{cases}
} \quad \quad \quad \quad
$
${f_y = \frac{1}{4}\mathrm{e}^{-\frac{y}{4}} ,\quad y \in \left(0, +\infty \right) }
$

Графики плотности (PDF) данных распределений изображены ниже: